In [6]:
!pip install nltk

You should consider upgrading via the '/home/raul/kge/bin/python3.7 -m pip install --upgrade pip' command.


In [32]:
import numpy as np
import pandas as pd
import itertools as it

import math
import joblib
from gensim.parsing.preprocessing import STOPWORDS

import pickle
#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy

#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale

import h2o

import pyLDAvis
#import pyLDAvis.gensim  # don't skip this

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import HdpModel


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp


from sklearn.model_selection import *
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
pd.__version__

'1.3.4'

In [27]:
df = joblib.load('../../corpus/processed/bbc_coref_resolved_cleaned')
df.head(3)

,news,type,coref_text,cleaned,entidades
0,China had role in Yukos split-up\n \n China le...,business,China had role in Yukos split-up\n \n China le...,china role yukos split china lend russia $ 6bn...,"{Rosneft, 27bn, last year, Reports, Aton, Yuko..."
1,Oil rebounds from weather effect\n \n Oil pric...,business,Oil rebounds from weather effect\n \n Oil pric...,oil rebound weather effect oil price recover a...,"{42.30, 1.33, Monday, last week, Achilles, yea..."
2,Indonesia 'declines debt freeze'\n \n Indonesi...,business,Indonesia 'declines debt freeze'\n \n Indonesi...,indonesia decline debt freeze indonesia longer...,"{India, Indonesia, Seychelles, 1.2bn, AP, abou..."


In [28]:
df.columns = ['texts','category','coref_text','cleaned','entities']

In [33]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
pipeline = ["lemmatizer", "tagger", "parser", "ner"]

texts = df.cleaned
docs = list(nlp.pipe(texts))


# Tags I want to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'SYM']
tokens = []
for summary in docs:
    proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.lemma_.lower() not in STOPWORDS and token.is_alpha]
    tokens.append(proj_tok)
    
    
tokens_without_sw = tokens


id2word = corpora.Dictionary(tokens_without_sw)


# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tokens_without_sw]



# Gesnim

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
import re
import nltk
from pprint import pprint
#textos  = [nltk.word_tokenize(sent) for sent in df.cleaned]
from gensim.parsing.preprocessing import STOPWORDS

In [ ]:
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.1))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.15))
beta.append('symmetric')


In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step, alpha, beta):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    model_parameters = []
    for num_topics in range(start, limit, step):
        for a in alpha:
            for b in beta:
                
                model  = LdaModel(corpus=corpus,
                           id2word=id2word,
                           num_topics=num_topics, 
                           random_state=0,
                           chunksize=20,
                           alpha=a,
                                  eta = b,
                           per_word_topics=True)

                model_list.append(model)
                coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
                coherence_values.append(coherencemodel.get_coherence())
                model_parameters.append({'num_topics':num_topics,'alpha':a,'eta':b})

    return model_list, coherence_values, model_parameters

In [12]:
%%time
model_list, coherence_values,model_parameters = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=tokens_without_sw, start=2, limit=20, step=1,alpha = alpha, beta = beta)


CPU times: user 1h 54min 26s, sys: 35min 8s, total: 2h 29min 35s
Wall time: 2h 7min 38s


In [13]:
import seaborn as sns

In [14]:
df = pd.DataFrame(model_parameters)

In [15]:
df['coherence'] = coherence_values
df.head()

,num_topics,alpha,eta,coherence
0,2,0.01,0.01,0.401696
1,2,0.01,0.16,0.522137
2,2,0.01,0.31,0.540668
3,2,0.01,0.46,0.543676
4,2,0.01,0.61,0.544922


In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [17]:
df ['comparacion'] = 'num_topics'+str(df['num_topics'])+'_alpha_'+str(df['alpha'])+'_eta_'+str(df['eta'])

In [12]:
df.loc[df.coherence>0.55,:]

,num_topics,alpha,eta,coherence,comparacion
12,2,0.11,0.61,0.550275,num_topics0 2\n1 2\n2 2\n...
110,3,0.11,0.91,0.567110,num_topics0 2\n1 2\n2 2\n...
117,3,0.21,0.76,0.560509,num_topics0 2\n1 2\n2 2\n...
118,3,0.21,0.91,0.559464,num_topics0 2\n1 2\n2 2\n...
125,3,0.31,0.76,0.560509,num_topics0 2\n1 2\n2 2\n...
126,3,0.31,0.91,0.563643,num_topics0 2\n1 2\n2 2\n...
133,3,0.41,0.76,0.551734,num_topics0 2\n1 2\n2 2\n...
134,3,0.41,0.91,0.562728,num_topics0 2\n1 2\n2 2\n...
141,3,0.51,0.76,0.556595,num_topics0 2\n1 2\n2 2\n...
142,3,0.51,0.91,0.555435,num_topics0 2\n1 2\n2 2\n...


In [14]:
df.loc[((df.num_topics==7) & (df.coherence>=0.55)),:]

,num_topics,alpha,eta,coherence,comparacion
508,7,0.31,0.61,0.551582,num_topics0 2\n1 2\n2 2\n...


In [15]:
modelo = model_list[508] #antiguo 243

In [21]:
joblib.dump(model_list,'lda_model_list_bbc')
#joblib.dump(modelo,'bbc_topic_modeling_final_gesnim')
joblib.dump(df,'topics_evaluation_models')

In [34]:
model_list = joblib.load('lda_model_list_bbc')
df = joblib.load('topics_evaluation_models')

In [40]:
len(id2word)

20337

In [35]:
import pyLDAvis
import pyLDAvis.gensim_models


pyLDAvis.enable_notebook()#0.299680
p = pyLDAvis.gensim_models.prepare(modelo, corpus, id2word)
p

IndexError: index 20335 is out of bounds for axis 1 with size 20335

In [ ]:
modelo.get_topic_terms(1,10)

In [ ]:
reverse_map = dict((modelo.id2word[id],id) for id in modelo.id2word)

In [20]:
topics = modelo.show_topics(formatted = False, num_words=100)

In [21]:
joblib.dump(topics,'new_bbc_topics_7_n')

['new_bbc_topics_7_n']

In [61]:
len(topics)

7

In [22]:
diccionario={}
vocabulario = set()

for topic in topics:
    lista_subdocumentos = []
    diccionario1 = dict()
    for tupla in topic[1]:
        vocabulario.add (tupla[0])
        diccionario1[tupla[0]]=tupla[1]
        lista_subdocumentos.append({tupla[0]:tupla[1]})
    diccionario[topic[0]]=diccionario1
        

In [23]:
joblib.dump(diccionario,'new_bbc_topics_7_n')

['new_bbc_topics_7_n']

In [24]:
joblib.dump(vocabulario,'vocabulario_7_n')

['vocabulario_7_n']

/home/raul/kge/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/raul/kge/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/raul/kge/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/raul/kge/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/raul/kge/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav

In [ ]:
columnas = [0,1,2,3,4,5,6]
df = pd.DataFrame(columns=['0','1','2','3','4','5','6'])
cont = 0
for documento in modelo.get_document_topics(corpus):
    visitados = []
    listado_valores = []
    print ('visitando el doc '+str(cont))
    print (documento)
    for tupla in documento:
        actual = tupla[0]# mejor coger min y max y rellenar con 0
        print('actual '+str(actual))
        if (len(visitados)>0):
            last_visited = visitados[len(visitados)-1]
        else:
            last_visited = -1
        visitados.append(actual)
        visitados.sort()
        print('ultimo '+str(last_visited))

        if actual == 0  or (actual>0 and actual - last_visited==1):
            listado_valores.append(tupla[1])
        elif actual>0 and actual - last_visited!=1:
            for i in range(last_visited+1,actual):
                visitados.append(i)
                listado_valores.append(0)
                visitados.sort()
            listado_valores.append(tupla[1])
    last_visited = visitados[len(visitados)-1]
    print(visitados)
    if 6 not in visitados:
        
        for i in range(last_visited+1,7):
            visitados.append(i)
            listado_valores.append(0)
    cont = cont + 1
    print(listado_valores)
    df.loc[len(df)] = listado_valores
    
 

In [42]:
df = df.loc[:,[column for column in df.columns if column!='target']]

In [43]:
df.head()

,0,1,2,3,4,5,6
0,0.278883,0.306827,0.184531,0.096925,0.020748,0.096622,0.015462
1,0.078109,0.258998,0.000000,0.108336,0.151951,0.000000,0.396373
2,0.402490,0.219638,0.201257,0.054296,0.000000,0.000000,0.119026
3,0.084805,0.304821,0.296181,0.137818,0.171239,0.000000,0.000000
4,0.054294,0.287917,0.222955,0.256412,0.044131,0.000000,0.132676


In [ ]:
topics = []

for indx,x in df.iterrows():
    print(x)
    topics.append(np.argmax(x))

df['new_topic']=topics

In [45]:
df.head()

,0,1,2,3,4,5,6,new_topic
0,0.278883,0.306827,0.184531,0.096925,0.020748,0.096622,0.015462,1
1,0.078109,0.258998,0.000000,0.108336,0.151951,0.000000,0.396373,6
2,0.402490,0.219638,0.201257,0.054296,0.000000,0.000000,0.119026,0
3,0.084805,0.304821,0.296181,0.137818,0.171239,0.000000,0.000000,1
4,0.054294,0.287917,0.222955,0.256412,0.044131,0.000000,0.132676,1


In [73]:
joblib.dump(df,'doc_topic_matrix_bbc_7')

['doc_topic_matrix_bbc_7']

In [57]:
df2 = joblib.load('../../corpus/processed/bbc_coref_resolved_cleaned')
df2.head()

,news,type,coref_text,cleaned,entidades
0,China had role in Yukos split-up\n \n China lent Russia $6bn (Â£3....,business,China had role in Yukos split-up\n \n China lent Russia $6bn (Â£3....,china role yukos split china lend russia $ 6bn â£3.2bn help russia...,"{Rosneft, 27bn, last year, Reports, Aton, Yukos, Sergei Oganesyan,..."
1,Oil rebounds from weather effect\n \n Oil prices recovered in Asia...,business,Oil rebounds from weather effect\n \n Oil prices recovered in Asia...,oil rebound weather effect oil price recover asian trade tuesday f...,"{42.30, 1.33, Monday, last week, Achilles, year-ago, Paris, more t..."
2,Indonesia 'declines debt freeze'\n \n Indonesia no longer needs th...,business,Indonesia 'declines debt freeze'\n \n Indonesia no longer needs th...,indonesia decline debt freeze indonesia longer need debt freeze of...,"{India, Indonesia, Seychelles, 1.2bn, AP, about $48bn, Aceh provin..."
3,"$1m payoff for former Shell boss\n \n Shell is to pay $1m (Â£522,0...",business,"$1m payoff for former Shell boss\n \n Shell is to pay $1m (Â£522,0...","$ 1 m payoff shell boss shell pay $ 1 m â£522,000 ex finance chief...","{$1m, Ms Boynton, Shell, Â£82.7, Judy Boynton, 2003, Walter van, J..."
4,US bank in $515m SEC settlement\n \n Five Bank of America subsidia...,business,The SEC have agreed to pay a total of $515m (Â£277m) to settle an ...,sec agree pay total $ 515 m â£277 m settle investigation fraudulen...,"{between $50-100,000, last year, the last two years, Columbia Mana..."


In [59]:
df['target']=df2.type
df['target'] = df['target'].astype('category')

In [60]:
df.shape

(2224, 9)

In [61]:
df.head()

,0,1,2,3,4,5,6,new_topic,target
0,0.278883,0.306827,0.184531,0.096925,0.020748,0.096622,0.015462,1,business
1,0.078109,0.258998,0.000000,0.108336,0.151951,0.000000,0.396373,6,business
2,0.402490,0.219638,0.201257,0.054296,0.000000,0.000000,0.119026,0,business
3,0.084805,0.304821,0.296181,0.137818,0.171239,0.000000,0.000000,1,business
4,0.054294,0.287917,0.222955,0.256412,0.044131,0.000000,0.132676,1,business


In [62]:
df.columns = ['0','1','2','3','4','5','6','new_topic','target']
df.to_csv('./datatopic_bbc_7.csv',index=False)

In [63]:
from h2o.estimators import H2OXGBoostEstimator, H2ODeepLearningEstimator,H2ORandomForestEstimator,H2OSupportVectorMachineEstimator
h2o.init(max_mem_size='14G')
frame = h2o.upload_file('./datatopic_bbc_7.csv',sep=',',header=1)


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 mins 39 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,1 month and 11 days
H2O_cluster_name:,H2O_from_python_raul_plh8li
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.41 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [64]:
frame['new_topic']=frame['new_topic'].asfactor()
columnas =list( df.columns )
columnas.remove('target')
columnas.remove('new_topic')


In [65]:
frame.head()

0,1,2,3,4,5,6,new_topic,target
0.278883,0.306827,0.184531,0.0969249,0.0207485,0.0966225,0.0154618,1,business
0.078109,0.258998,0,0.108336,0.151951,0,0.396373,6,business
0.40249,0.219638,0.201257,0.0542962,0,0,0.119026,0,business
0.0848049,0.304821,0.296181,0.137818,0.171239,0,0,1,business
0.0542936,0.287917,0.222955,0.256412,0.0441312,0,0.132676,1,business
0.241299,0.387742,0.325691,0.0385116,0,0,0,1,business
0.194151,0.24836,0,0.291127,0,0,0.257216,3,business
0.0139721,0.301273,0,0.673688,0,0,0,3,business
0.176582,0.407558,0.135474,0.218718,0,0,0.0578034,1,business
0.0944281,0.242334,0.0137358,0,0.600167,0,0.0450052,4,business


In [66]:

from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch

search_criteria={'strategy':"RandomDiscrete",
                'max_models':15,
                'seed':4241}

# Rango de hiper-parámetros para grid search 9 combinaciones 
hyper_params = {'sample_rate':[0.4,0.5,0.6]  , 
                'col_sample_rate':[0.4,0.5,0.6],
                'min_rows':[30,50],
               'max_depth':[10,30,100],
               'ntrees':[10,20,30,100,150],
                'reg_lambda':[0,0.2,0.5,0.7,1] ,
                'reg_alpha':[0,0.2,0.5,0.7,1] } 

xgb_rand_grid=H2OGridSearch(
#             H2OGradientBoostingEstimator(
                H2OXGBoostEstimator(
                model_id='gbm_rand_grid',
                seed=665,
                # ntrees=150,
                nfolds=5, #2,
                stopping_metric='mean_per_class_error',
                stopping_rounds=10,
                score_tree_interval=1
                
            ),
    search_criteria=search_criteria,
    hyper_params=hyper_params)

      
xgb_rand_grid.train(x = columnas,y='target',training_frame=frame)

xgboost Grid Build progress: |███████████████████████████████████████████████████| (done) 100%
      col_sample_rate  max_depth  min_rows  ntrees  reg_alpha  reg_lambda  \
0                 0.6       10.0      30.0    50.0        0.0         0.5   
1                 0.4      100.0      30.0    34.0        0.2         1.0   
2                 0.6      100.0      30.0    20.0        0.7         1.0   
3                 0.6      100.0      30.0    30.0        0.7         0.2   
4                 0.5       30.0      50.0    53.0        0.5         0.2   
5                 0.6       30.0      30.0    20.0        0.5         0.7   
6                 0.6      100.0      50.0    30.0        0.5         0.0   
7                 0.5       30.0      50.0    30.0        0.2         0.5   
8                 0.5       10.0      50.0    20.0        0.7         0.2   
9                 0.4       30.0      50.0    40.0        0.0         0.7   
10                0.5      100.0      50.0    20.0        

In [67]:
xgb_rand_grid[0]

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  Grid_XGBoost_py_3_sid_a316_model_python_1648453972353_2_model_13


Model Summary: 


,,number_of_trees
0,,50.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.15381403337858682
RMSE: 0.39219132241622434
LogLoss: 0.4838038500171792
Mean Per-Class Error: 0.18009224682861708
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,business,entertainment,politics,sport,tech,Error,Rate
0,412.0,12.0,60.0,2.0,24.0,0.192157,98 / 510
1,17.0,308.0,25.0,23.0,13.0,0.202073,78 / 386
2,52.0,23.0,317.0,2.0,23.0,0.239808,100 / 417
3,3.0,22.0,0.0,486.0,0.0,0.048924,25 / 511
4,15.0,31.0,40.0,1.0,313.0,0.217500,87 / 400
5,499.0,396.0,442.0,514.0,373.0,0.174460,388 / 2.224



Top-5 Hit Ratios: 


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.771007,0.013859,0.791011,0.776316,0.770925,0.755102,0.761682
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.228993,0.013859,0.208989,0.223684,0.229075,0.244898,0.238318
3,err_count,101.800000,5.495453,93.000000,102.000000,104.000000,108.000000,102.000000
4,logloss,0.615710,0.043065,0.545776,0.608835,0.627473,0.637584,0.658880
5,max_per_class_error,0.326544,0.029491,0.287234,0.366197,0.313953,0.340659,0.324675
6,mean_per_class_accuracy,0.765413,0.014314,0.788608,0.761683,0.768627,0.753804,0.754342
7,mean_per_class_error,0.234587,0.014314,0.211392,0.238317,0.231373,0.246196,0.245658
8,mse,0.195786,0.012143,0.176217,0.192334,0.201188,0.202295,0.206895
9,pr_auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-03-28 09:55:46,11.533 sec,0.0,0.800000,1.609438,0.770234,NaN,NaN
1,,2022-03-28 09:55:46,11.539 sec,1.0,0.733664,1.338107,0.350270,NaN,NaN
2,,2022-03-28 09:55:46,11.543 sec,2.0,0.671474,1.139910,0.254047,NaN,NaN
3,,2022-03-28 09:55:46,11.547 sec,3.0,0.623119,1.007906,0.246403,NaN,NaN
4,,2022-03-28 09:55:46,11.551 sec,4.0,0.584981,0.914691,0.243705,NaN,NaN
5,,2022-03-28 09:55:46,11.555 sec,5.0,0.550361,0.833216,0.234263,NaN,NaN
6,,2022-03-28 09:55:46,11.560 sec,6.0,0.524476,0.774847,0.229766,NaN,NaN
7,,2022-03-28 09:55:46,11.564 sec,7.0,0.503443,0.728426,0.226619,NaN,NaN
8,,2022-03-28 09:55:46,11.569 sec,8.0,0.487418,0.694341,0.223921,NaN,NaN
9,,2022-03-28 09:55:46,11.574 sec,9.0,0.473574,0.663590,0.219424,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,6,1550.709595,1.000000,0.248973
1,0,1427.341553,0.920444,0.229166
2,1,911.890076,0.588047,0.146408
3,2,727.096436,0.468880,0.116738
4,5,678.545349,0.437571,0.108943
5,4,596.125366,0.384421,0.095710
6,3,336.714355,0.217136,0.054061


In [68]:
xgb_rand_grid[0].save_mojo('bbc_evaluation_topics_model')

'/home/raul/doctorado/notebooks/semantic_oboe/notebooks/bbc/bbc_evaluation_topics_model/Grid_XGBoost_py_3_sid_a316_model_python_1648453972353_2_model_13.zip'